## 1 환경설정

In [54]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>"))
%matplotlib inline

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import warnings
# import folium
# import folium.plugins
import requests
import json
# import folium.plugins
from IPython.display import Image
warnings.filterwarnings('ignore')

mpl.rcParams['figure.figsize'] = (12,8)  #시각화 figure default 설정
mpl.rcParams['font.family'] = 'NanumGothic' #폰트 디폴트 설정
mpl.rcParams['font.size'] = 10    #폰트 사이즈 디폴트 설정
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format='retina' # 그래프 글씨 뚜렷
pd.options.display.float_format = '{:.3f}'.format # 넘파이 과학적 표기 제거

## 2 전처리 작업

In [55]:
%pwd

'c:\\Users\\LeeSeungYong\\myproject\\[공모전] (신한카드상) 2023년 문화관광 데이터분석대회\\tourism\\방문자수추이'

In [56]:
import os
import zipfile

total = pd.DataFrame(columns = ["기준년월", "기초지자체", "방문자 구분", "방문자 수", "지역"])

for dirname, _, filenames in os.walk('c:\\Users\\LeeSeungYong\\myproject\\[공모전] (신한카드상) 2023년 문화관광 데이터분석대회\\tourism\\방문자수추이'):
    for filename in filenames:
        if filename.endswith(".zip"):
            zip_path = os.path.join(dirname, filename)

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                csv_files = zip_ref.namelist()

                for csv_file in csv_files:
                   if csv_file.endswith('.csv'):
                       with zip_ref.open(csv_file) as file:
                            df = pd.read_csv(file, encoding = 'cp949')
                            name = filename.split("_")[1]
                            df["지역"] = [name] * df.shape[0]
                            total = pd.concat([total,df], axis = 0, ignore_index = True)

In [57]:
total

,기준년월,기초지자체,방문자 구분,방문자 수,지역
0,2018,중구,현지인방문자(a),24474720.000,인천시+중구
1,2018,중구,외지인방문자(b),55750030.000,인천시+중구
2,2018,중구,외국인방문자(c),4677649.000,인천시+중구
3,2018,중구,외부방문자(b+c),60427679.000,인천시+중구
4,2018,중구,전체방문자(a+b+c),84902399.000,인천시+중구
...,...,...,...,...,...
1195,2022,이천시,현지인방문자(a),52185144.000,이천시
1196,2022,이천시,외지인방문자(b),40014681.000,이천시
1197,2022,이천시,외국인방문자(c),32876.000,이천시
1198,2022,이천시,외부방문자(b+c),40047557.000,이천시


In [58]:
for_total = total[total["방문자 구분"] == "외국인방문자(c)"].copy()

In [59]:
pivot_total = pd.pivot_table(data = for_total, index = "지역", columns = "기준년월", values = "방문자 수").reset_index()
# pivot_total["2019성장율"] = ((pivot_total[2019] / pivot_total[2018]) - 1) * 100

In [60]:
pivot_total[["시도", "시군구"]] = pivot_total["지역"].str.split("+", expand = True)

In [61]:
pivot_total

combine_list = ["성남시", "수원시", "안양시", "용인시", "전주시", "창원시", "천안시", "청주시", "포항시"]

rest = pivot_total[~pivot_total['시도'].isin(combine_list)]
combine = pivot_total[pivot_total['시도'].isin(combine_list)]

In [62]:
combine = pd.DataFrame(combine.groupby("시도").sum()).reset_index()

In [63]:
# rest["시군구"] = rest["지역"].apply(lambda x : x.split("+")[0])
rest["시군구"] = rest["지역"].apply(lambda x : x.split("+")[1] if len(x.split("+")) >=2 else x.split("+")[0])

In [64]:
combine = combine[[2018, 2019, 2020, 2021, 2022, '시도']]
combine.columns = [2018, 2019, 2020, 2021, 2022, '시군구']

In [65]:
rest.head(3)

기준년월,지역,2018,2019,2020,2021,2022,시도,시군구
0,강릉시,418154.000,161141.000,20965.000,15905.000,38761.000,강릉시,강릉시
1,거제시,372140.000,429561.000,100307.000,116698.000,126530.000,거제시,거제시
2,경주시,353033.000,515873.000,45216.000,41119.000,53423.000,경주시,경주시


In [66]:
combine.head(3)

,2018,2019,2020,2021,2022,시군구
0,558681.000,642532.000,123559.000,118680.000,166539.000,성남시
1,988980.000,1274307.000,155625.000,120570.000,187650.000,수원시
2,219776.000,304854.000,43706.000,41678.000,56161.000,안양시


In [67]:
merge = pd.concat([rest, combine], axis = 0)

In [68]:
merge

,지역,2018,2019,2020,2021,2022,시도,시군구
0,강릉시,418154.000,161141.000,20965.000,15905.000,38761.000,강릉시,강릉시
1,거제시,372140.000,429561.000,100307.000,116698.000,126530.000,거제시,거제시
2,경주시,353033.000,515873.000,45216.000,41119.000,53423.000,경주시,경주시
3,광주시+서구,115197.000,162655.000,24980.000,24440.000,22257.000,광주시,서구
4,구미시,178516.000,165604.000,85210.000,131755.000,46469.000,구미시,구미시
5,군산시,209028.000,342279.000,72743.000,90378.000,84832.000,군산시,군산시
6,대구시+동구,191635.000,328404.000,28692.000,23573.000,31687.000,대구시,동구
7,대구시+수성구,131638.000,172264.000,25030.000,28673.000,32845.000,대구시,수성구
8,대구시+중구,199109.000,311266.000,24751.000,13010.000,21921.000,대구시,중구
9,대전시+유성구,192121.000,230968.000,34281.000,30443.000,52475.000,대전시,유성구


In [69]:
merge = merge[["시군구", 2018, 2019, 2020, 2021, 2022]].reset_index(drop = True)

In [70]:
merge
merge["2019성장율"] = ((merge[2019] / merge[2018]) - 1) * 100
merge["2020성장율"] = ((merge[2020] / merge[2019]) - 1) * 100
merge["2021성장율"] = ((merge[2021] / merge[2020]) - 1) * 100
merge["2022성장율"] = ((merge[2022] / merge[2021]) - 1) * 100

In [71]:
merge

,시군구,2018,2019,2020,2021,2022,2019성장율,2020성장율,2021성장율,2022성장율
0,강릉시,418154.000,161141.000,20965.000,15905.000,38761.000,-61.464,-86.990,-24.135,143.703
1,거제시,372140.000,429561.000,100307.000,116698.000,126530.000,15.430,-76.649,16.341,8.425
2,경주시,353033.000,515873.000,45216.000,41119.000,53423.000,46.126,-91.235,-9.061,29.923
3,서구,115197.000,162655.000,24980.000,24440.000,22257.000,41.197,-84.642,-2.162,-8.932
4,구미시,178516.000,165604.000,85210.000,131755.000,46469.000,-7.233,-48.546,54.624,-64.731
5,군산시,209028.000,342279.000,72743.000,90378.000,84832.000,63.748,-78.747,24.243,-6.136
6,동구,191635.000,328404.000,28692.000,23573.000,31687.000,71.370,-91.263,-17.841,34.421
7,수성구,131638.000,172264.000,25030.000,28673.000,32845.000,30.862,-85.470,14.555,14.550
8,중구,199109.000,311266.000,24751.000,13010.000,21921.000,56.329,-92.048,-47.436,68.493
9,유성구,192121.000,230968.000,34281.000,30443.000,52475.000,20.220,-85.158,-11.196,72.371


In [72]:
a = pd.read_excel("시도 순위.xlsx")
concat = pd.merge(left = a, right = merge, how = "inner", left_on = "가맹점(시군구)", right_on = "시군구")
concat.drop([0,3],axis = 0, inplace = True)

In [73]:
concat.to_excel("전체통합2022추가.xlsx", index = False)

In [74]:
concat

,가맹점(시도),가맹점(시군구),취급액sum,취급액mean,이용건수sum,이용건수mean,자연관광,역사관광,문화관광,숙박관광,...,시군구,2018,2019,2020,2021,2022,2019성장율,2020성장율,2021성장율,2022성장율
1,인천,중구,223729422824,28374054.892,430123,54.550,4.500,0.100,3.200,10.300,...,중구,4677649.000,7041546.000,956396.000,610262.000,1402258.000,50.536,-86.418,-36.191,129.780
2,인천,연수구,69314228518,11901481.545,205083,35.213,0.100,0.400,13.500,9.700,...,연수구,512341.000,590006.000,115709.000,113057.000,222196.000,15.159,-80.389,-2.292,96.534
4,강원,속초시,39955525499,14362158.698,98844,35.530,10.700,0.500,4.100,24.300,...,속초시,202988.000,258692.000,15026.000,5373.000,34326.000,27.442,-94.192,-64.242,538.861
5,경북,경주시,25367293521,7077927.880,95986,26.782,2.700,16.700,12.000,20.000,...,경주시,353033.000,515873.000,45216.000,41119.000,53423.000,46.126,-91.235,-9.061,29.923
6,강원,강릉시,35462836012,18056433.815,91184,46.428,13.700,2.600,7.600,19.100,...,강릉시,418154.000,161141.000,20965.000,15905.000,38761.000,-61.464,-86.990,-24.135,143.703
7,경기,성남시,63102148528,17499209.242,89839,24.914,0.133,0.667,9.100,8.467,...,성남시,558681.000,642532.000,123559.000,118680.000,166539.000,15.009,-80.770,-3.949,40.326
8,경기,수원시,59258124787,12457036.953,88895,18.687,0.575,2.450,8.375,9.550,...,수원시,988980.000,1274307.000,155625.000,120570.000,187650.000,28.851,-87.787,-22.525,55.636
9,대구,수성구,11852899146,4263632.786,75452,27.141,4.600,0.400,10.200,12.100,...,수성구,131638.000,172264.000,25030.000,28673.000,32845.000,30.862,-85.470,14.555,14.550
10,대전,유성구,19810638950,5626424.013,68716,19.516,0.100,0.500,9.800,11.000,...,유성구,192121.000,230968.000,34281.000,30443.000,52475.000,20.220,-85.158,-11.196,72.371
11,경기,화성시,23453321792,7870242.212,65037,21.824,6.800,2.000,8.600,6.900,...,화성시,564933.000,770990.000,190198.000,205014.000,218333.000,36.475,-75.331,7.790,6.497


In [75]:
concat["성장율순위"] = concat["2019성장율"].rank(ascending = True)